In [1]:
from __future__ import print_function, division

import numpy as np
import pandas as pd
import json, os, csv, shutil
from glob import glob
import matplotlib.pyplot as plt

In [2]:
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Lambda, Activation, Flatten, Input
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop, SGD
from keras.utils import np_utils

import h5py
import cv2
import PIL
from PIL import Image

/Users/swapnilpote/.virtualenvs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/swapnilpote/.virtualenvs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
path = 'data/'

In [4]:
batch_size = 32

# Data preprocessing

In [5]:
dir_names = os.listdir(os.path.join(path, 'train/'))
dir_names

['NoF', 'OTHER', 'ALB', 'DOL', 'BET', 'LAG', 'YFT', 'SHARK']

In [6]:
dir_dict = {}

for i in range(len(dir_names)):
    dir_dict[dir_names[i]] = i

In [7]:
dir_dict

{'ALB': 2,
 'BET': 4,
 'DOL': 3,
 'LAG': 5,
 'NoF': 0,
 'OTHER': 1,
 'SHARK': 7,
 'YFT': 6}

In [8]:
X_train = []
y_train = []
trn_size = []
trn_filename = []

In [ ]:
for folder in os.listdir(path + 'train/'):
    for file in os.listdir(path + 'train/' + folder + '/'):
        if os.path.isfile(path + 'train/' + folder + '/' + file):
            
            input_img = cv2.imread(path + 'train/' + folder + '/' + file)
            height, width, channel = input_img.shape
            input_img = cv2.resize(input_img, (224, 224))
            X_train.append(input_img)
            
            y_cat = np_utils.to_categorical(dir_dict[folder], 8) # 8 represent number of class
            y_train.append(y_cat)
            
            trn_size.append([width, height])
            trn_filename.append(file)

In [ ]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(trn_size[0:5])
print(trn_filename[0:5])

In [ ]:
y_train[466]

In [ ]:
X_valid = []
y_valid = []
val_size = []
val_filename = []

In [ ]:
for folder in os.listdir(path + 'valid/'):
    for file in os.listdir(path + 'valid/' + folder + '/'):
        if os.path.isfile(path + 'valid/' + folder + '/' + file):
            
            input_img = cv2.imread(path + 'valid/' + folder + '/' + file)
            height, width, channel = input_img.shape
            input_img = cv2.resize(input_img, (224, 224))
            X_valid.append(input_img)
            
            y_cat = np_utils.to_categorical(dir_dict[folder], 8) # 8 represent number of class
            y_valid.append(y_cat)
            
            val_size.append([width, height])
            val_filename.append(file)

In [ ]:
X_valid = np.asarray(X_valid)
y_valid = np.asarray(y_valid)

In [ ]:
print(X_valid.shape)
print(y_valid.shape)
print(val_size[0:5])
print(val_filename[0:5])

In [ ]:
y_valid[0]

# BB

In [ ]:
import ujson as json

In [ ]:
anno_classes = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft']

In [ ]:
bb_json = {}

for c in anno_classes:
    j = json.load(open('{}annos/{}_labels.json'.format(path, c), 'r'))
    
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations']) > 0:
            bb_json[l['filename']] = l['annotations'][-1]

In [ ]:
bb_json['img_07763.jpg']

In [ ]:
empty_bbox = {'height': 0., 'width': 0., 'x': 0., 'y': 0.}

In [ ]:
for f in trn_filename:
    if not f in bb_json.keys(): bb_json[f] = empty_bbox
for f in val_filename:
    if not f in bb_json.keys(): bb_json[f] = empty_bbox

In [ ]:
trn_resize_dim = []
val_resize_dim = []

In [ ]:
def convert_bb(img, width, height):
    bb = []
    conv_x = (224. / width)
    conv_y = (224. / height)
    bb.append(bb_json[img]['height'] * conv_y)
    bb.append(bb_json[img]['width'] * conv_x)
    bb.append(max(bb_json[img]['x'] * conv_x, 0))
    bb.append(max(bb_json[img]['y'] * conv_y, 0))
    return bb

In [ ]:
len(bb_json.keys())

In [ ]:
trn_bbox = []
val_bbox = []

In [ ]:
for i in range(len(trn_filename)):
    trn_bbox.append(convert_bb(trn_filename[i], trn_size[i][0], trn_size[i][1]))

In [ ]:
for i in range(len(val_filename)):
    val_bbox.append(convert_bb(val_filename[i], val_size[i][0], val_size[i][1]))

In [ ]:
trn_bbox = np.asarray(trn_bbox)
val_bbox = np.asarray(val_bbox)

In [ ]:
print(trn_bbox[-1200])
print(val_bbox[-600])

# NN

In [ ]:
input_img = Input(shape=(224, 224, 3))

x = Conv2D(32, (3, 3), padding = 'same', activation = 'relu')(input_img)
# x = Conv2D(32, (3, 3), padding = 'same', activation = 'relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), padding = 'same', activation = 'relu')(x)
# x = Conv2D(64, (3, 3), padding = 'same', activation = 'relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dense(512, activation = 'relu')(x)
# x = Dense(512, activation = 'relu')(x)

x_bb = Dense(4, name='bb')(x)
x_class = Dense(8, activation='softmax', name='class')(x)

model = Model([input_img], [x_bb, x_class])
model.compile(Adam(lr=0.001), loss=['mse', 'categorical_crossentropy'], metrics=['accuracy'],
             loss_weights=[.001, 1.])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, [trn_bbox, y_train], batch_size=batch_size, nb_epoch=3, 
             validation_data=(X_valid, [val_bbox, y_valid]))